In [11]:
# Top of the file
import os
import pickle

import torch
from torch.utils.data import DataLoader

import vae_model.build

from data.preset import DexedPresetsParams
import vae_config
import data.build

Can't import sounddevice!!!!


## Load model

In [12]:
config_dir = './vae_model'

# Create model
with open(os.path.join(config_dir, 'subset_samplers.pickle'), 'rb') as f:
    idx_helper = pickle.load(f)
with open(os.path.join(config_dir, 'config_train.pickle'), 'rb') as f:
    conf_train = pickle.load(f)
with open(os.path.join(config_dir, 'config_model.pickle'), 'rb') as f:
    conf_model = pickle.load(f)
    
model_config, train_config = vae_config.model, vae_config.train
dataset = data.build.get_dataset(model_config, train_config)

_, _, _, model = vae_model.build.build_extended_ae_model(conf_model, conf_train,
                                                                 idx_helper)
model = model.eval()

# Load from checkpoint
checkpoint_state_dict = torch.load(
    os.path.join(config_dir, '00133.tar'), 
    map_location=torch.device('cpu')
)

model.load_state_dict(checkpoint_state_dict['ae_model_state_dict'])

ae_model, reg_model = model.ae_model, model.reg_model

[PresetDataset] Cannot open '/home/ubuntu/environment/project/audio-synth-rl/data/stats/DexedDataset_spectrogram_nfft1024hop0256mels0257.json' stats file.
[PresetDataset] No pre-computed spectrogram stats can be found. No normalization will be performed
[PresetIndexesHelper] 144 learnable VSTi parameters, learnable tensor representation size: 610
FORCE BIGGER ENC/DEC NETWORKS = False
MIDI NOTES = ((60, 100),)
self.spectrogram_input_size (257, 347)


In [13]:
initial_ground_truth_spectrograms = torch.randn(1, 1, 257, 345)

In [14]:
# Convert our dataset spectrogram to spectrograms compatible with the VAE
processed_initial_ground_truth_spectrograms = torch.cat([initial_ground_truth_spectrograms, torch.zeros(initial_ground_truth_spectrograms.size(0), 1, 257, 2)], axis=-1)
processed_initial_ground_truth_spectrograms.shape

torch.Size([1, 1, 257, 347])

In [19]:
# Use VAE to get learnable param outputs for initial spectrograms
def get_full_synth_params_from_vae(input_spectrograms):
    with torch.no_grad():
        # Use correct values here
        preset_UID = 0
        ref_midi_pitch = 60
        ref_midi_velocity = 100

        sample_info = torch.tensor([preset_UID, ref_midi_pitch, ref_midi_velocity], dtype=torch.int32).unsqueeze(0)
        ae_out = ae_model(input_spectrograms, sample_info)  # Spectral VAE - tuple output

        _, _, z_K_sampled, _, _ = ae_out
        v_out = reg_model(z_K_sampled)

        out_presets_instance = DexedPresetsParams(learnable_presets=v_out, dataset=dataset)
        synth_params_tensor = out_presets_instance.get_full()
        return synth_params_tensor

In [21]:
get_full_synth_params_from_vae(processed_initial_ground_truth_spectrograms).shape

torch.Size([1, 155])